In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset
import matplotlib.pyplot as plt
from torch.utils.data import Dataset

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
x_train = pd.read_csv('/kaggle/input/ncr/train.csv')
x_test = pd.read_csv('/kaggle/input/ncr/test.csv')
submit = pd.read_csv('/kaggle/input/ncr/sample_submission.csv')

In [ ]:
print(x_test)

In [ ]:
#Sliceing Test dataset


x_label = x_train.loc[:,'label']
x_image = x_train.drop('label',axis=1)
test_label = x_test


In [ ]:
#Change CSV to Tensor

x_train_np = np.reshape(x_image,(-1,1,28,28))
print(x_train_np.shape)
x_train_tenser = torch.Tensor(x_train_np)
x_train_tenser = x_train_tenser.float()
y_train_tenser = torch.Tensor(x_label)
y_train_tenser = y_train_tenser.long()



print(x_train_tenser[0])
print(x_train_tenser[0].shape)


In [ ]:
x_test_np = np.reshape(x_test,(-1,1,28,28))
print(x_test_np.shape)
x_test_tensor = torch.Tensor(x_test_np)

In [ ]:
class My_Number_set(Dataset): 
  def __init__(self,x_data,y_data):
    self.x_data = torch.FloatTensor(x_data)
    self.y_data = torch.LongTensor(y_data)

  # 총 데이터의 개수를 리턴
  def __len__(self): 
    return len(self.y_data)

  # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
  def __getitem__(self, idx): 
    x = self.x_data[idx]
    y = self.y_data[idx] 
    return x, y
  

In [ ]:
#Tenser test
TRAIN_MNIST = My_Number_set(x_data=x_train_tenser,y_data=y_train_tenser)

print(TRAIN_MNIST[4])



In [ ]:
#MNIST image example
images_and_labels = TRAIN_MNIST

for index in range(20): # 5개의 샘플만 출력
    image ,label = images_and_labels[index]
    
    plt.subplot(4, 5, index + 1)
    plt.axis('off')
    plt.imshow((image.view([28,28])).numpy(), cmap=plt.cm.gray_r, interpolation='nearest')
    plt.title('sample: %i' % label)

del index, image, label

In [ ]:
batch_size = 100
data_loader = torch.utils.data.DataLoader(dataset=TRAIN_MNIST,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

torch.manual_seed(777)
learning_rate = 0.02
training_epochs = 30

In [ ]:
class CNN_arch2(torch.nn.Module):
    def __init__(self):
        super(CNN_arch2, self).__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU())

        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Dropout(0.25))


        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=2, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=1),
            torch.nn.Dropout(0.25))
        
        self.fc = torch.nn.Linear(12544, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        x_normal = x/torch.max(x)
        out = self.layer1(x_normal)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)   # 전결합층을 위해서 Flatten
        out = self.fc(out)

        return out

In [ ]:
model = CNN_arch2()
criterion = torch.nn.CrossEntropyLoss()    # 비용 함수에 소프트맥스 함수 포함되어져 있음.
optimizer = torch.optim.Adagrad(model.parameters(), lr=learning_rate)


In [ ]:
total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))


In [ ]:
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader: 
        X = X
        Y = Y

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))


In [ ]:
with torch.no_grad():
    X_test = x_test_tensor

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1)

In [ ]:
print(correct_prediction[0])


for index in range(20): # 5개의 샘플만 출력
    image = x_test_tensor[index]
    label = correct_prediction[index]
    
    plt.subplot(4, 5, index + 1)
    plt.axis('off')
    plt.imshow((image.view([28,28])).numpy(), cmap=plt.cm.gray_r, interpolation='nearest')
    plt.title('sample: %i' % label)

del index, image, label

In [ ]:
df = pd.DataFrame(data=correct_prediction)
submit['Label'] = np.array(correct_prediction)
submit


In [ ]:
submit.to_csv('submission.csv',index=False)
print("Your submission was successfully saved!")